In [7]:
#importing neccesary files

from ipynb.fs.defs.jaccard_coefficient import remove_punc, remove_art_connector, remove_emoji
import nltk
import re
from tqdm import tqdm
import math
import numpy as np
import pickle as pkl
import joblib
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from collections import Counter
nltk.download('stopwords')
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/frostrot/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Filtering

In [8]:
def stemming(text):
    x = text.split(" ")
    ps = PorterStemmer()
    return " ".join([ps.stem(i) for i in x])

In [9]:
#Remove stopwords, and convert shorted words into there extended forms

def stopword(text):
  EXTENDED_FORMS = {"aren't": 'are not', "can't": 'cannot', "couldn't": 'could not', "didn't": 'did not', "doesn't": 'does not', "don't": 'do not', "hadn't": 'had not', "hasn't": 'has not', "haven't": 'have not', "he'd": 'he would', "he'll": 'he will', "he's": 'he is', "i'd": 'i would', "i'll": 'i will', "i'm": 'i am', "isn't": 'is not', "it's": 'it is', "it'll": 'it will', "i've": 'i have', "let's": 'let us', "mightn't": 'might not', "mustn't": 'must not',"n't": 'not', "shan't": 'shall not', "she'd": 'she would', "she'll": 'she will', "she's": 'she is', "shouldn't": 'should not', "that's": 'that is', "there's": 'there is', "they'd": 'they would', "they'll": 'they will', "they're": 'they are', "they've": 'they have', "we'd": 'we would', "we're": 'we are', "weren't": 'were not', "we've": 'we have', "what'll": 'what will', "what're": 'what are', "what's": 'what is', "what've": 'what have', "where's": 'where is', "who'd": 'who would', "who'll": 'who will', "who're": 'who are', "who's": 'who is', "who've": 'who have', "won't": 'will not', "wouldn't": 'would not', "you'd": 'you would', "you'll": 'you will', "you're": 'you are', "you've": 'you have', "'re": ' are', "wasn't": 'was not', "we'll": 'we will', "'cause": 'because', "could've": 'could have', "how'd": 'how did', "how'd'y": 'how do you', "how'll": 'how will', "how's": 'how is', "I'd": 'I would', "I'd've": 'I would have', "I'll": 'I will', "I'll've": 'I will have', "I'm": 'I am', "I've": 'I have', "i'd've": 'i would have', "i'll've": 'i will have', "it'd": 'it would', "it'd've": 'it would have', "it'll've": 'it will have', "ma'am": 'madam', "mayn't": 'may not', "might've": 'might have', "mightn't've": 'might not have', "must've": 'must have', "mustn't've": 'must not have', "needn't": 'need not', "needn't've": 'need not have', "o'clock": 'of the clock', "oughtn't": 'ought not', "oughtn't've": 'ought not have', "sha'n't": 'shall not', "shan't've": 'shall not have', "she'd've": 'she would have', "she'll've": 'she will have', "should've": 'should have', "shouldn't've": 'should not have', "so've": 'so have', "so's": 'so as', "this's": 'this is', "that'd": 'that would', "that'd've": 'that would have', "there'd": 'there would', "there'd've": 'there would have', "here's": 'here is', "they'd've": 'they would have', "they'll've": 'they will have', "to've": 'to have', "we'd've": 'we would have', "we'll've": 'we will have', "what'll've": 'what will have', "when's": 'when is', "when've": 'when have', "where'd": 'where did', "where've": 'where have', "who'll've": 'who will have', "why's": 'why is', "why've": 'why have', "will've": 'will have', "won't've": 'will not have', "would've": 'would have', "wouldn't've": 'would not have', "y'all": 'you all', "y'all'd": 'you all would', "y'all'd've": 'you all would have', "y'all're": 'you all are', "y'all've": 'you all have', "you'd've": 'you would have', "you'll've": 'you will have'}
  x= word_tokenize(text)
  for i in range(len(x)):
    if x[i] in EXTENDED_FORMS:
      x[i] = EXTENDED_FORMS[x[i]]
    if x[i] in stopwords.words('english'):
      x[i]=''
  x=remove_punc(x)
  x=remove_art_connector(x)
  return " ".join(x)

In [10]:
#Filter the parsed text, by, converting them into lowercase, removing any tags, extra spaces.
vocab = set()
def filter(item,b):
  global vocab
  if type(item)==str:
    item=item.lower()
    item=re.sub('[#@]\w+\s*',"",item)
    item=re.sub(r'\\N','',item)
    item=remove_emoji(item)
    item=stopword(item)
    item=stemming(item)
    item = word_tokenize(item)
    item = Counter(item)
    if b:
      for x in item.keys():
        vocab.add(x)
  return item

In [11]:
with open('./pickle_files/data.pkl','rb') as f:
    data = pkl.load(f)

In [12]:
for files in tqdm(data):
    files['filtered_content'] = filter(files['content'],True)

100%|██████████| 1133/1133 [06:06<00:00,  3.09it/s]


In [13]:
len(vocab)

71002

In [14]:
# Dumping in Pickle File

pkl.dump(data,open('./pickle_files/data_2.pkl','wb'))
pkl.dump(vocab,open('./pickle_files/vocab.pkl','wb'))

### Loading data

In [15]:
with open('./pickle_files/data_2.pkl','rb') as f:
    data = pkl.load(f)

with open('./pickle_files/vocab.pkl','rb') as f:
    vocab = pkl.load(f)

### Creating posting list

In [16]:
nested_posting_list = {}

for files in tqdm(data):
    for words in files['filtered_content']:
        if words in nested_posting_list:
            if files['file'] in nested_posting_list[words]:
                nested_posting_list[words][files['file']]+=1
            else:
                nested_posting_list[words][files['file']]=1
        else:
            nested_posting_list[words]={}
            nested_posting_list[words][files['file']]=1

100%|██████████| 1133/1133 [00:00<00:00, 3175.51it/s]


In [17]:
idf = {}
total_documents = len(data)

for word in nested_posting_list:
    idf[word] = math.log(total_documents/len(nested_posting_list[word])+1)

In [18]:
corpus_matrix = np.zeros((total_documents,5,len(vocab)))

vocab = list(vocab)

for i,doc in tqdm(enumerate(data)):
    for j,word in enumerate(vocab):
        tf0 = 0
        tf1 = 0
        tf2 = 0
        tf3 = 0
        tf4 = 0.5
        if doc['file'] in nested_posting_list[word]:
            tf0 = 1
            tf1 = nested_posting_list[word][doc['file']]
            tf2 = tf1/sum(doc['filtered_content'].values())
            tf3 = math.log(1+tf1)
            tf4 = 0.5+0.5*(tf1/max(doc['filtered_content'].values()))

        corpus_matrix[i][0][j] = tf0*idf[word]
        corpus_matrix[i][1][j] = tf1*idf[word]
        corpus_matrix[i][2][j] = tf2*idf[word]
        corpus_matrix[i][3][j] = tf3*idf[word]
        corpus_matrix[i][4][j] = tf4*idf[word]


1133it [05:51,  3.22it/s]


In [19]:
joblib.dump(corpus_matrix,open('./pickle_files/tf_idf_corpus.pkl','wb'),compress=9)

### Quering


In [20]:
with open('./pickle_files/tf_idf_corpus.pkl','rb') as f:
    corpus_matrix = joblib.load(f)

In [21]:
def cosine(x,y):
    return x@y/((np.sum(x**2)**0.5)*(np.sum(y**2)**0.5))

In [23]:
n = int(input("Enter the number of queries"))

for _ in range(n):
    query = input("Enter the query sentence")
    query_posting_list = {}
    cleaned_query = filter(query,False)

    query_doc = {'file':'query'}
    query_doc['filtered_content'] = cleaned_query
    for value in cleaned_query:
        query_posting_list[value] = {"query":cleaned_query[value]}
    print(query_posting_list)

    query_vector = np.zeros((1,5,len(vocab)))

    for i,doc in enumerate([query_doc]):
        for j,word in enumerate(vocab):
            tf0 = 0
            tf1 = 0
            tf2 = 0
            tf3 = 0
            tf4 = 0.5
            if word in query_posting_list:
                tf0 = 1
                tf1 = query_posting_list[word]['query']
                tf2 = tf1/sum(query_doc['filtered_content'].values())
                tf3 = math.log(1+tf1)
                tf4 = 0.5+0.5*(tf1/max(query_doc['filtered_content'].values()))

            query_vector[i][0][j] = tf0*idf[word]
            query_vector[i][1][j] = tf1*idf[word]
            query_vector[i][2][j] = tf2*idf[word]
            query_vector[i][3][j] = tf3*idf[word]
            query_vector[i][4][j] = tf4*idf[word]
    
    cosine_sim = {}

    for i in range(len(corpus_matrix)):
        cosine_sim[i]={}

        cosine_sim[i][0] = cosine(query_vector[0][0],corpus_matrix[i][0])
        cosine_sim[i][1] = cosine(query_vector[0][1],corpus_matrix[i][1])
        cosine_sim[i][2] = cosine(query_vector[0][2],corpus_matrix[i][2])
        cosine_sim[i][3] = cosine(query_vector[0][3],corpus_matrix[i][3])
        cosine_sim[i][4] = cosine(query_vector[0][4],corpus_matrix[i][4])
        # print(cosine_sim[i])
    
    tf0 = sorted(cosine_sim.items(),key=lambda x: x[1][0],reverse=True)
    tf1 = sorted(cosine_sim.items(),key=lambda x: x[1][1],reverse=True)
    tf2 = sorted(cosine_sim.items(),key=lambda x: x[1][2],reverse=True)
    tf3 = sorted(cosine_sim.items(),key=lambda x: x[1][3],reverse=True)
    tf4 = sorted(cosine_sim.items(),key=lambda x: x[1][4],reverse=True)

    # print(tf1)

    mapping = {0:"For Binary TF Weight",1:"For Raw Count TF Weight",2:"For TF Weight",3:"For Binary Log Normalization Weight",4:"For Double Normalization TF Weight"}
    tf_mapping = {0:tf0,1:tf1,2:tf2,3:tf3,4:tf4}

    for i in range(5):
        print(mapping[i])
        for j in range(5):
            print(data[tf_mapping[i][j][0]]['file'],end="- ")
            print(tf_mapping[i][j][1][i])
        print("---------------------")


        
    
    

{'love': {'query': 1}, 'eat': {'query': 1}, 'pizza': {'query': 1}}
For Binary TF Weight
caramels.des- 0.10125059109476077
strsdiet.txt- 0.08785655110198817
who.txt- 0.079241579032555
fbipizza.txt- 0.07324150269229059
focaccia.brd- 0.06840880059190871
---------------------
For Raw Count TF Weight
caramels.des- 0.10125059109476077
strsdiet.txt- 0.08785655110198817
who.txt- 0.079241579032555
fbipizza.txt- 0.07324150269229059
focaccia.brd- 0.06840880059190871
---------------------
For TF Weight
caramels.des- 0.10125059109476077
strsdiet.txt- 0.08785655110198819
who.txt- 0.07924157903255499
fbipizza.txt- 0.0732415026922906
focaccia.brd- 0.06840880059190872
---------------------
For Binary Log Normalization Weight
caramels.des- 0.10125059109476078
strsdiet.txt- 0.08785655110198817
who.txt- 0.07924157903255499
fbipizza.txt- 0.0732415026922906
focaccia.brd- 0.06840880059190871
---------------------
For Double Normalization TF Weight
aids.txt- 0.999997657659433
bitnet.txt- 0.9999976575771227
jo